In [19]:
import abc
import logging
import random
import numpy as np
import torch
from torch.autograd import Variable

class Algorithm(metaclass=abc.ABCMeta):
    def __init__(self, module_name, name, seed, details=False):
        self.logger = logging.getLogger(module_name)
        self.name = name
        self.seed = seed
        self.details = details
        self.prediction_details = {}
        self.history = {}

        if self.seed is not None:
            random.seed(seed)
            np.random.seed(seed)

    def __str__(self):
        return self.name

    @abc.abstractmethod
    def fit(self, X):
        """
        Train the algorithm on the given dataset
        """

    @abc.abstractmethod
    def predict(self, X):
        """
        :return anomaly score
        """


class PyTorchUtils(metaclass=abc.ABCMeta):
    def __init__(self, seed, gpu):
        self.gpu = gpu
        self.seed = seed
        if self.seed is not None:
            torch.manual_seed(self.seed)
            torch.cuda.manual_seed(self.seed)
        self.framework = 0

    @property
    def device(self):
        return torch.device('cuda:{self.gpu}' if torch.cuda.is_available() and self.gpu is not None else 'cpu')

    def to_var(self, t, **kwargs):
        # ToDo: check whether cuda Variable.
        t = t.to(self.device)
        return Variable(t, **kwargs)

    def to_device(self, model):
        model.to(self.device)

In [20]:
import time
start_time = time.time()

import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from scipy.stats import multivariate_normal
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import trange
import matplotlib.pyplot as plt
%matplotlib inline

!git clone https://github.com/priyojitk/mit-resources.git yahoo_dataset
file_path = '/content/yahoo_dataset/'

import sys
sys.path.append(file_path)
from sklearn.metrics import confusion_matrix

fatal: destination path 'yahoo_dataset' already exists and is not an empty directory.


In [21]:

class LSTMEDModule(nn.Module):
    def __init__(self, n_features: int, hidden_size: int, n_layers: tuple, dropout: tuple):
        super().__init__()
        self.n_features = n_features
        self.hidden_size = hidden_size
        self.num_lyers = n_layers
        self.dropout = dropout

        self.encoder = nn.LSTM(self.n_features, self.hidden_size, batch_first=True, num_layers=self.num_lyers[0], dropout=self.dropout[0], bias = True)
        self.decoder = nn.LSTM(self.hidden_size, self.n_features, batch_first=True, num_layers=self.num_lyers[1], dropout=self.dropout[1], bias = True)

    def forward(self, input):
        sequence_length = input.shape[1]
     
        encoded, (last_hidden, _) = self.encoder(input.float())
        decoder_input = encoded[:, -1:].repeat(1, sequence_length, 1)
        decoded, _ = self.decoder(decoder_input)  

        return decoded

In [22]:
class LSTMED(Algorithm, PyTorchUtils):
    def __init__(self, name: str = 'LSTM-ED',
                 num_epochs: int = 10, 
                 batch_size: int = 20, 
                 lr: float = 1e-3,
                 hidden_size: int = 5,
                 sequence_length: int = 48,
                 train_gaussian_percentage: float = 0.2,
                 n_layers: tuple = (1, 1), 
                 use_bias: tuple = (True, True),
                 dropout: tuple = (0, 0),
                 seed: int = 1,
                 gpu: int = None,
                 details=True):
        
        Algorithm.__init__(self, __name__, name, seed, details=details)

        if self.seed is not None:
            random.seed(seed)
            np.random.seed(seed)
        
        
        PyTorchUtils.__init__(self, seed, gpu)
        
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.lr = lr

        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        self.train_gaussian_percentage = train_gaussian_percentage

        self.n_layers = n_layers
        self.use_bias = use_bias
        self.dropout = dropout
        self.mean, self.cov = None, None

        # self.lstmed = LSTMEDModule(1, self.hidden_size, self.n_layers, self.dropout)


    def fit(self, X: pd.DataFrame):
        
        X.interpolate(inplace=True) #
        X.bfill(inplace=True) #backward fill values
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(data.shape[0] - self.sequence_length + 1)]
        indices = np.random.permutation(len(sequences)) 
        
        split_point = int(self.train_gaussian_percentage * len(sequences))
        train_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, drop_last=True,
                                  sampler=SubsetRandomSampler(indices[:-split_point]), pin_memory=True)
        train_gaussian_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, drop_last=True,
                                           sampler=SubsetRandomSampler(indices[-split_point:]), pin_memory=True)

        self.lstmed = LSTMEDModule(X.shape[1], self.hidden_size, self.n_layers, self.dropout)
        self.to_device(self.lstmed)
        optimizer = torch.optim.Adam(self.lstmed.parameters(), lr=self.lr)

        self.lstmed.train() #set to training mode=True
        self.history['train_loss'] = []
        self.history['val_loss'] = []
        
        for epoch in trange(self.num_epochs): #self.num_epochs
            self.logger.debug(f'Epoch {epoch+1}/{self.num_epochs}.')
            self.lstmed.train() #set to training mode=True
            train_batch_loss = []
            for ts_batch in train_loader:
                output = self.lstmed(self.to_var(ts_batch))
                loss = nn.MSELoss(reduction='mean')(output, self.to_var(ts_batch.float())) 
                self.lstmed.zero_grad()
                loss.backward()
                optimizer.step()
                train_batch_loss.append(loss.item())
            train_loss = sum(train_batch_loss)/len(train_batch_loss)
            self.history['train_loss'].append(train_loss)
        

            self.lstmed.eval()
            val_batch_loss = []
            for ts_batch in train_gaussian_loader:
                output = self.lstmed(self.to_var(ts_batch))
                val_loss = nn.MSELoss(reduction='mean')(output, self.to_var(ts_batch.float())) 
                val_batch_loss.append(val_loss.item())
            self.history['val_loss'].append(sum(val_batch_loss)/len(val_batch_loss))

        self.lstmed.eval()
        error_vectors = []
        count = 0
        for ts_batch in train_gaussian_loader:
            output = self.lstmed(self.to_var(ts_batch))
            error = nn.L1Loss(reduction='none')(output, self.to_var(ts_batch.float())) #MAE
            error_vectors += list(error.reshape(-1, X.shape[1]).data.cpu().numpy())
            
        self.mean = np.mean(error_vectors, axis=0)
        self.cov = np.cov(error_vectors, rowvar=False)

    def predict(self, X: pd.DataFrame):
        X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(data.shape[0] - self.sequence_length + 1)]
        data_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, shuffle=False, drop_last=False)

        self.lstmed.eval()
        mvnormal = multivariate_normal(self.mean, self.cov, allow_singular=True)
        scores = []
        outputs = []
        errors = []
        for idx, ts in enumerate(data_loader):
            output = self.lstmed(self.to_var(ts))
            error = nn.L1Loss(reduction='none')(output, self.to_var(ts.float()))
            score = -mvnormal.logpdf(error.reshape(-1, X.shape[1]).data.cpu().numpy())
            scores.append(score.reshape(ts.size(0), self.sequence_length)) #ts.size() == batch size
            if self.details:
                outputs.append(output.data.numpy())
                errors.append(error.data.numpy())

        # stores seq_len-many scores per timestamp and averages them
        scores = np.concatenate(scores)
        lattice = np.full((self.sequence_length, data.shape[0]), np.nan)
        for i, score in enumerate(scores):
            lattice[i % self.sequence_length, i:i + self.sequence_length] = score
        scores = np.nanmean(lattice, axis=0)

        return scores

In [23]:
from sklearn.metrics import accuracy_score, fbeta_score
from sklearn.metrics import precision_recall_fscore_support as prf
from sklearn.metrics import roc_curve, auc, roc_auc_score

def get_accuracy_precision_recall_fscore(y_true: list, y_pred: list):
        accuracy = accuracy_score(y_true, y_pred)
        # warn_for=() avoids log warnings for any result being zero
        precision, recall, f_score, _ = prf(y_true, y_pred, average='binary', warn_for=())
        if precision == 0 and recall == 0:
            f01_score = 0
        else:
            f01_score = fbeta_score(y_true, y_pred, average='binary', beta=0.1)
        return accuracy, precision, recall, f_score, f01_score

# accuracy, precision, recall, f_score, f01_score = get_accuracy_precision_recall_fscore(_test['is_anomaly'], pred_y)

In [24]:
def threshold(score):
    return np.nanmean(score) + 2 * np.nanstd(score)
def get_specificity(actual_y, pred_y):
    tn, fp, fn, tp = confusion_matrix(actual_y, pred_y).ravel()
    specificity = tn / (tn + fp)
    return specificity

In [25]:
avg_accuracy, avg_precision, avg_recall, avg_f_score, avg_f01_score = 0, 0, 0, 0, 0
avg_auc = 0
avg_specificity = 0
A1 = 67
no_of_dataset = A1
zero_anomaly_test = 0

In [26]:
# model = LSTMED(num_epochs=30, sequence_length=10, hidden_size=25, n_layers=(2, 2), dropout=(0, 0)) #57 708 recall 618
# pytorch_total_params = sum(p.numel() for p in model.lstmed.parameters() if p.requires_grad)
# print(pytorch_total_params)

# pytorch_total_params = sum(p.numel() for p in model.lstmed.parameters())
# print(pytorch_total_params)

In [27]:
for ind in range(1, no_of_dataset + 1):

    print('\nDataset ', ind)
    logging.info(f'\n\n=============\n Dataset {ind} : \n=============')
    
    df = pd.read_csv(f'{file_path}dataset/ydata-labeled-time-series-anomalies-v1_0/A1Benchmark/real_{ind}.csv', index_col='timestamp')

    from sklearn.preprocessing import StandardScaler
    df['value'] = df['value'].astype(np.float64)

    # Normalize the  data (center around 0 and scale to remove the variance).
    scaler = StandardScaler()
    df['value'] = scaler.fit_transform(df['value'].values.reshape(-1, 1))

    len_of_data = df.shape[0]
    _train = df[0:int(len_of_data * 0.4)] # 40 percent
    _test = df[int(len_of_data * 0.4):] # 60 percent
    print('length of training set :', len(_train))
    print('length of test set :', len(_test))  
    
    #If there is no anomaly in 'test' data or whole data ignore the dataset
    anomaly_count_test = len(_test[_test['is_anomaly'] == 1])
    print('#anomaly in test set :', anomaly_count_test)
    if anomaly_count_test == 0:
        zero_anomaly_test += 1
        continue
    
    logging.info(f'length of training set : {len(_train)}')
    logging.info(f'length of test set : {len(_test)}')

    # model = LSTMED(num_epochs=30, sequence_length=10, hidden_size=10, n_layers=(1, 1), dropout=(0, 0)) #575 p 71 recall61
    model = LSTMED(num_epochs=30, sequence_length=10, hidden_size=25, n_layers=(2, 2), dropout=(0, 0)) #57 708 recall 618


    model.fit(_train[['value']].copy())

    # plt.figure(figsize=(20, 4))
    # plt.title('Training loss / Val Loss')
    # plt.plot(model.history['train_loss'], label='train_loss')
    # plt.plot(model.history['val_loss'], label='val_loss')
    # plt.legend()
    # plt.show()

    scores = model.predict(_test[['value']].copy())
    th = threshold(scores)
    pred_y = np.where(np.array(scores) > th, 1, 0) 
    # th = 0.05
    # pred_y = np.where(np.array(scores) < th, 1, 0) 

    # plt.figure(figsize=(20, 4))
    # plt.title('Pred_y / actual_anomaly')
    # plt.plot(pred_y, label='pred_y')
    # plt.plot(_test['is_anomaly'].values, label='actual_anomaly')
    # plt.legend()
    # plt.show()

    accuracy, precision, recall, f_score, f01_score = get_accuracy_precision_recall_fscore(_test['is_anomaly'], pred_y)
    auc = roc_auc_score(_test['is_anomaly'], pred_y)
    specificity = get_specificity(_test['is_anomaly'], pred_y)

    
    print(f'Accuracy : {accuracy}')
    print(f'Precision : {precision}')
    print(f'recall : {recall}')
    print(f'f_score : {f_score}')
    print(f'f01_score : {f01_score}')
    print(f'auc : {auc}')
    print(f'specificity : {specificity}')

    avg_accuracy += accuracy
    avg_precision += precision
    avg_recall += recall
    avg_f_score += f_score
    avg_f01_score += f01_score
    avg_auc += auc
    avg_specificity += specificity


denominator = (no_of_dataset - zero_anomaly_test)
print(f'\nAvg Accuracy : {avg_accuracy / denominator }')
print(f'Avg Precision : {avg_precision / denominator }')
print(f'Avg recall : {avg_recall / denominator }')
print(f'Avg f_score : {avg_f_score / denominator }')
print(f'Avg f01_score : {avg_f01_score / denominator }')
print(f'Avg auc : {avg_auc / denominator }')
print(f'Avg specificity : {avg_specificity / denominator }')

print(f"--- {(time.time() - start_time)/60} minutes ---" )


  0%|          | 0/30 [00:00<?, ?it/s]


Dataset  1
length of training set : 568
length of test set : 852
#anomaly in test set : 2


  0%|          | 0/30 [00:00<?, ?it/s]

Accuracy : 0.9870892018779343
Precision : 0.15384615384615385
recall : 1.0
f_score : 0.2666666666666667
f01_score : 0.15514592933947774
auc : 0.9935294117647059
specificity : 0.9870588235294118

Dataset  2
length of training set : 575
length of test set : 864
#anomaly in test set : 16


  0%|          | 0/30 [00:00<?, ?it/s]

...

Accuracy : 0.9777777777777777
Precision : 0.4166666666666667
recall : 0.29411764705882354
f_score : 0.3448275862068966
f01_score : 0.4149548069022186
auc : 0.6428822125508915
specificity : 0.9916467780429594

Dataset  66
length of training set : 569
length of test set : 855
#anomaly in test set : 21


  0%|          | 0/30 [00:00<?, ?it/s]

Accuracy : 0.9976608187134502
Precision : 1.0
recall : 0.9047619047619048
f_score : 0.9500000000000001
f01_score : 0.9989588755856326
auc : 0.9523809523809523
specificity : 1.0

Dataset  67
length of training set : 569
length of test set : 854
#anomaly in test set : 23


100%|██████████| 30/30 [00:07<00:00,  3.94it/s]


Accuracy : 0.9918032786885246
Precision : 1.0
recall : 0.6956521739130435
f_score : 0.8205128205128205
f01_score : 0.995686999383857
auc : 0.8478260869565217
specificity : 1.0

Avg Accuracy : 0.9719128519140602
Avg Precision : 0.7084813910701936
Avg recall : 0.6182131014252652
Avg f_score : 0.5743589757635398
Avg f01_score : 0.7009270743114889
Avg auc : 0.8042421051804566
Avg specificity : 0.9902711089356475
--- 7.833815745512644 minutes ---
